In [1]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
from math import isnan
from itertools import combinations
import Kruskal_v2 as Kv
import sys

In [ ]:
def em_algorithm(seed_val, samples, num_clusters, max_num_iter=100):
    """
    This function is for the EM algorithm.
    :param seed_val: Seed value for reproducibility. Type: int
    :param samples: Observed x values. Type: numpy array. Dimensions: (num_samples, num_nodes)
    :param num_clusters: Number of clusters. Type: int
    :param max_num_iter: Maximum number of EM iterations. Type: int
    :return: loglikelihood: Array of log-likelihood of each EM iteration. Type: numpy array.
                Dimensions: (num_iterations, ) Note: num_iterations does not have to be equal to max_num_iter.
    :return: topology_list: A list of tree topologies. Type: numpy array. Dimensions: (num_clusters, num_nodes)
    :return: theta_list: A list of tree CPDs. Type: numpy array. Dimensions: (num_clusters, num_nodes, 2)

    You can change the function signature and add new parameters. Add them as parameters with some default values.
    i.e.
    Function template: def em_algorithm(seed_val, samples, k, max_num_iter=10):
    You can change it to: def em_algorithm(seed_val, samples, k, max_num_iter=10, new_param_1=[], new_param_2=123):
    """

    # Set the seed
    np.random.seed(seed_val)
    # TODO: Implement EM algorithm here.
    # Start: Example Code Segment. Delete this segment completely before you implement the algorithm.
    #### randomly create trees
    print("Running EM algorithm...") 
    loglikelihood = []
    # for iter_ in range(max_num_iter):
    #     loglikelihood.append(np.log((1 + iter_) / max_num_iter))
    from Tree import TreeMixture
    tm = TreeMixture(num_clusters=num_clusters, num_nodes = samples.shape[1])
    tm.simulate_pi(seed_val=seed_val)
    tm.simulate_trees(seed_val=seed_val)
    tm.sample_mixtures(num_samples=samples.shape[0], seed_val = seed_val)
    topology_list = []
    theta_list = []

    for i in range(num_clusters):
        topology_list.append(tm.clusters[i].get_topology_array())

        theta_list.append(tm.clusters[i].get_theta_array())



    topology_list = np.array(topology_list)
    theta_list = np.array(theta_list)
    pi         = np.ones((1,num_clusters))
    pi         = pi / np.sum(pi)

    #### start do EM algorithm
    n_samples = samples.shape[0]
    ### responsibility 
    ### 1. calculate p(sample|tk,thetak)
    for i in range(max_num_iter):
        r = np.zeros((n_samples,num_clusters))
        for n in range(n_samples):
            for c in range(num_clusters):
                r[n,c] = calculate_likelihood(samples[n],topology_list[c],theta_list[c]) + sys.float_info.epsilon
            #r[n,:] = r[n,:]/ np.sum(r[n,:])

    ### 2. responsibility
        r =  pi * (r / np.sum(r, axis = 1).reshape((n_samples, 1)))

        loglikelihood.append(np.sum(np.log(np.sum(r, axis = 1))))
    ### 3. pi
        pi = np.mean(r,axis = 0)
        pi = pi / np.sum(pi)
        #print(r)
    ### 4. compute weight
        NoN = list(combinations([i for i in range(samples.shape[1])],2))
        topology_list = []
        theta_list = []
        #g = [Kv.Graph(samples.shape[1])] * num_clusters
        for k in range(num_clusters):

            edges = []
            for e in NoN:
                # edge = (node1 , node2)
                nodeWedge = ComputeWeight(e, r, k, samples)
                if nodeWedge != 0:
                    #print((e[0], e[1], nodeWedge))
                    edges.append((e[0], e[1], nodeWedge))
                    #g[k].addEdge(e[0], e[1], nodeWedge)
            graphs = generate_graph(samples.shape[1],edges)
            #g[k].KruskalMST()
            result = Kv.maximum_spanning_tree(graphs)
            #result = g[k].maximum_spanning_tree()
            #print(result)
            new_topology = Find_New_topology(result, samples.shape[1])
            topology_list.append(new_topology)
            ### new theta
            new_theta = Compute_theta(r , k , new_topology , samples.shape[1],samples)
            theta_list.append(new_theta)

        theta_list = np.array(theta_list)
        topology_list = np.array(topology_list)
        if abs(loglikelihood[i] - loglikelihood[i-1]) < sys.float_info.epsilon and i != 0:
            loglikelihood = np.array(loglikelihood)
            return loglikelihood, topology_list, theta_list


    return loglikelihood, topology_list, theta_list


In [2]:
def main():
    # Code to process command line arguments
#     parser = argparse.ArgumentParser(description='EM algorithm for likelihood of a tree GM.')
#     parser.add_argument('--sample_filename', type=str, default = '/Users/candacechou/Desktop/homework/advML/Assignment2/AdvML19-master/2_5/data/q_2_5_tm_10node_20sample_4clusters.pkl_samples.txt',
#                         help='Specify the name of the sample file (i.e data/example_samples.txt)')
#     parser.add_argument('--output_filename', type=str, default = '/Users/candacechou/Desktop/homework/advML/Assignment2/AdvML19-master/2_5/result.txt',
#                         help='Specify the name of the output file (i.e data/example_results.txt)')
#     parser.add_argument('--num_clusters', type=int,default = 4, help='Specify the number of clusters (i.e 3)')
#     parser.add_argument('--seed_val', type=int, default= 42, help='Specify the seed value for reproducibility (i.e 42)')
#     parser.add_argument('--real_values_filename', type=str, default="",
#                         help='Specify the name of the real values file (i.e data/example_tree_mixture.pkl)')
    
    num_cluster = 4
    seed_val = 42
    # You can add more default parameters if you want.

    # print("Hello World!")
    # print("This file demonstrates the flow of function templates of question 2.5.")

    print("\n0. Load the parameters from command line.\n")

#     args = parser.parse_args()
#     print("\tArguments are: ", args)

    print("\n1. Load samples from txt file.\n")

    samples = np.loadtxt('/Users/candacechou/Desktop/homework/advML/Assignment2/AdvML19-master/2_5/data/q_2_5_tm_10node_20sample_4clusters.pkl_samples.txt', delimiter="\t", dtype=np.int32)
    num_samples, num_nodes = samples.shape
    # print("\tnum_samples: ", num_samples, "\tnum_nodes: ", num_nodes)
    # print("\tSamples: \n", samples)

    print("\n2. Run EM Algorithm.\n")

    loglikelihood, topology_array, theta_array = em_algorithm(seed_val, samples, num_clusters= num_clusters, max_num_iter= 100)
#     print(topology_array)
#     print(theta_array)
#     print("\n3. Save, print and plot the results.\n")

    save_results(loglikelihood, topology_array, theta_array, 'data/example_tree_mixture.pkl')

    # for i in range(args.num_clusters):
    #     print("\n\tCluster: ", i)
    #     print("\tTopology: \t", topology_array[i])
    #     print("\tTheta: \t", theta_array[i])

    plt.figure(figsize=(8, 3))
    plt.subplot(121)
    plt.plot(np.exp(loglikelihood), label='Estimated')
    plt.ylabel("Likelihood of Mixture")
    plt.xlabel("Iterations")
    plt.subplot(122)
    plt.plot(loglikelihood, label='Estimated')
    plt.ylabel("Log-Likelihood of Mixture")
    plt.xlabel("Iterations")
    plt.legend(loc=(1.04, 0))
    plt.show()

    print("\n4. Retrieve real results and compare.\n")
    if args.real_values_filename != "":
        print("\tComparing the results with real values...")

        print("\t4.1. Make the Robinson-Foulds distance analysis.\n")
        # TODO: Do RF Comparison

        print("\t4.2. Make the likelihood comparison.\n")
        # TODO: Do Likelihood Comparison


if __name__ == "__main__":
    main()



0. Load the parameters from command line.



usage: ipykernel_launcher.py [-h] [--sample_filename SAMPLE_FILENAME]
                             [--output_filename OUTPUT_FILENAME]
                             [--num_clusters NUM_CLUSTERS]
                             [--seed_val SEED_VAL]
                             [--real_values_filename REAL_VALUES_FILENAME]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/candacechou/Library/Jupyter/runtime/kernel-b0205e0a-6eaf-4d2d-af6d-deea166d769c.json


SystemExit: 2

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
